In [1]:
!pip install OpenAi
!pip install langchain
!pip install python-dotenv
!pip install langchain-openai
!pip install sqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.

---
Tools / TOOLS.py

In [2]:
import sqlite3
from pydantic.v1 import BaseModel
from typing import List
from langchain.tools import Tool

# Avoid Colab Output srollbar
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [5]:
conn = sqlite3.connect("db.sqlite")

def run_sqlite_query(query):
  c = conn.cursor()
  try:
    c.execute(query)
    return c.fetchall()
  except sqlite3.OperationalError as err:
    return f"The following error occurred: {str(err)}"

def list_tables():
  c = conn.cursor()
  c.execute("SELECT name FROM sqlite_master WHERE type='table';")
  rows = c.fetchall()
  return "\n".join(row[0] for row in rows if row[0] is not None)

def describe_tables(tables_names):
  c = conn.cursor()
  tables = ', '.join("'" + table + "'" for table in tables_names)
  rows = c.execute(f"SELECT sql FROM sqlite_master WHERE type='table' and name IN ({tables});")
  return '\n'.join(row[0] for row in rows if row[0] is not None)

# - - -

class RunQueryArgsSchema(BaseModel):
  # This is for improve ChatGPT understanding about each tool
  query: str

run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="Run a sqlite query.",
    func=run_sqlite_query,
    args_schema=RunQueryArgsSchema
)

class DescribeTablesArgsSchema(BaseModel):
   # This is for improve ChatGPT understanding about each tool
  tables_names: List[str]

describe_tables_tool = Tool.from_function(
    name="describe_tables",
    description="Given a list of tables names, return a schema of those.",
    func=describe_tables,
    args_schema=DescribeTablesArgsSchema
)

---
Tools / REPORT.py

In [4]:
from langchain.tools import StructuredTool
from pydantic.v1 import BaseModel

def write_report(filename, html):
  with open(filename, 'w') as f:
    f.write(html) # Whatever is provided by html file, Im going write in filename

class WriteReportArgsSchema(BaseModel):
  filename: str
  html: str

write_report_tool = StructuredTool.from_function(
    name="write_report",
    description="Write an HTML file to disk. Use this tool whenever someone asks for a report.",
    func=write_report,
    args_schema=WriteReportArgsSchema
)

---
Handlers / chat_model_start.py

In [24]:
from langchain_core.callbacks.base import BaseCallbackHandler

class ChatModelStartHandler():
  def on_chat_model_start(self, serialized, messages, **kwargs):
    print(messages)

---
MAIN.py

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)


from langchain.agents import OpenAIFunctionsAgent, AgentExecutor, create_openai_functions_agent
# from tools.py import my_function_tools, list_tables

In [25]:
load_dotenv(find_dotenv(), override=True)
os.environ.get('OPENAI_API_KEY')
os.environ.get('PINECONE_API_KEY')

handler = ChatModelStartHandler()


chatt = ChatOpenAI(
    #verbose=True,
    callbacks=[handler]
    )
#chatt = ChatOpenAI()

# Print the type of the callbacks argument
#print(type(chatt.callbacks))

# Print the type of the expected argument
#print(type(BaseCallbackManager))

ValidationError: 2 validation errors for ChatOpenAI
callbacks -> 0
  instance of BaseCallbackHandler expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCallbackHandler)
callbacks
  instance of BaseCallbackManager expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCallbackManager)

In [8]:
tables = list_tables()
print(tables)

prompt = ChatPromptTemplate(
    messages = [
        SystemMessage(content=(
            "You are an AI that has access to a SQLite database.\n"
            f"The database has tables of: {tables}\n"
            "Do not make any assumptions about what table exist "
            "Or what columns exist. Instead, use the 'describe_tables' function"
            )),
        MessagesPlaceholder(variable_name="chat_history"), #Always add this, before any human message
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

# MEMORY
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#List with all tools.
all_tools = [
    run_query_tool,
    describe_tables_tool,
    write_report_tool
    ]

# OpenAIFunctionsAgent
agent = OpenAIFunctionsAgent(
    llm=chatt,
    prompt=prompt,
    tools=all_tools
)

# # create_openai_functions_agent
# agent = create_openai_functions_agent(
#     chatt, all_tools, prompt
# )

agent_executor= AgentExecutor(
    agent=agent,
    #verbose=True,
    tools=all_tools,
    memory=memory

)

users
addresses
products
carts
orders
order_products


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.openai_functions_agent.base.OpenAIFunctionsAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_openai_functions_agent instead.
  warn_deprecated(


In [ ]:
#agent_executor("How many users are in the database?")
#agent_executor("How many users have provided a shipping address?")
agent_executor("Summarize 5 most popular products. Write the results to a report file.")

In [9]:
agent_executor(
    "How many orders are there? Write the result to an html report"
)

agent_executor(
    "Repeat the exact same procces for users"
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `{'query': 'SELECT COUNT(*) FROM orders'}`


[(1500,)]
Invoking: `write_report` with `{'filename': 'orders_report.html', 'html': '<h1>Number of Orders</h1><p>There are 1500 orders in the database.</p>'}`
responded: There are 1500 orders in the database. I will now write this result to an HTML report.

NoneThe number of orders has been written to an HTML report. You can download the report from the following link: [Download Orders Report](sandbox:/orders_report.html)

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `describe_tables` with `{'tables_names': ['users']}`


CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT UNIQUE,
    password TEXT
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT COUNT(*) FROM users'}`


[(2000,)]
Invoking: `write_report` with `{'filename': 'users_report.html', 'html': '<h1>Number of Users</h1><p>Total number of users: 2000</

{'input': 'Repeat the exact same procces for users',
 'chat_history': [HumanMessage(content='How many orders are there? Write the result to an html report'),
  AIMessage(content='The number of orders has been written to an HTML report. You can download the report from the following link: [Download Orders Report](sandbox:/orders_report.html)'),
  HumanMessage(content='Repeat the exact same procces for users'),
  AIMessage(content='The number of users has been written to an HTML report. You can download the report from the following link: [Download Users Report](sandbox:/users_report.html)')],
 'output': 'The number of users has been written to an HTML report. You can download the report from the following link: [Download Users Report](sandbox:/users_report.html)'}